In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import concat, lit

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Constructors_pipeline")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# load the results, drivers and constructors tables in dataframes
resultsDF = spark.read.format("csv").option("header", "true") \
       .load("gs://data-group1-ass2/results.csv")
driversDF = spark.read.format("csv").option("header", "true") \
       .load("gs://data-group1-ass2/drivers.csv")
constructorsDF = spark.read.format("csv").option("header", "true") \
       .load("gs://data-group1-ass2/constructors.csv")

# keep interesting columns
filteredResultsDF = resultsDF.select("resultId", "raceId", "driverId", "constructorId", "points")
filteredResultsDF.na.drop("any")
filteredDriversDF = driversDF.select("driverId", concat("forename",lit(" "),  "surname").alias("driver_name"), "nationality").withColumnRenamed("nationality", "driver_nationality")
filteredDriversDF.na.drop("any", subset = ["driverId"])
filteredConstructorsDF = constructorsDF.select("constructorId", "name", "nationality").withColumnRenamed("nationality", "constructor_nationality").withColumnRenamed("name", "constructor_name")
filteredConstructorsDF.na.drop("any", subset = ["constructorId"])

filteredResultsDF.show(5)
filteredDriversDF.show(5)
filteredConstructorsDF.show(5)

+--------+------+--------+-------------+------+
|resultId|raceId|driverId|constructorId|points|
+--------+------+--------+-------------+------+
|       1|    18|       1|            1|    10|
|       2|    18|       2|            2|     8|
|       3|    18|       3|            3|     6|
|       4|    18|       4|            4|     5|
|       5|    18|       5|            1|     4|
+--------+------+--------+-------------+------+
only showing top 5 rows

+--------+-----------------+------------------+
|driverId|      driver_name|driver_nationality|
+--------+-----------------+------------------+
|       1|   Lewis Hamilton|           British|
|       2|    Nick Heidfeld|            German|
|       3|     Nico Rosberg|            German|
|       4|  Fernando Alonso|           Spanish|
|       5|Heikki Kovalainen|           Finnish|
+--------+-----------------+------------------+
only showing top 5 rows

+-------------+----------------+-----------------------+
|constructorId|constructor_na

In [2]:
from pyspark.sql.functions import col
from pyspark.sql.types import NumericType, IntegerType

# Change points column to numeric type
filteredResultsDF.printSchema()
filteredResultsDF2 = filteredResultsDF.withColumn("points", col("points").cast(IntegerType()))

root
 |-- resultId: string (nullable = true)
 |-- raceId: string (nullable = true)
 |-- driverId: string (nullable = true)
 |-- constructorId: string (nullable = true)
 |-- points: string (nullable = true)



In [3]:
# Aggregate on driver and then constructor and add all points together
groupedResultsDF = filteredResultsDF2.groupby("driverId", "constructorId").sum("points")
# Then order by constructor and then points, to see the highest scoring drivers for every team
groupedResultsDF.orderBy(col("constructorId").asc(), col("sum(points)").desc()).show(20)

+--------+-------------+-----------+
|driverId|constructorId|sum(points)|
+--------+-------------+-----------+
|       1|            1|        913|
|      18|            1|        908|
|     117|            1|        458|
|     102|            1|        451|
|      14|            1|        412|
|      57|            1|        407|
|     846|            1|        378|
|       8|            1|        337|
|       4|            1|        241|
|     832|            1|        201|
|      77|            1|        135|
|     817|            1|        130|
|     182|            1|        128|
|     231|            1|        117|
|     187|            1|        109|
|     224|            1|        100|
|     304|            1|         94|
|      31|            1|         86|
|       5|            1|         75|
|     200|            1|         63|
+--------+-------------+-----------+
only showing top 20 rows



In [4]:
from pyspark.sql.window import Window
from pyspark.sql.functions import max, dense_rank

rankedDriverWindow = Window.partitionBy("constructorId").orderBy(col("sum(points)").desc())
denseRankPoints = dense_rank().over(rankedDriverWindow)
dfRankedDrivers = groupedResultsDF.select(col("driverId"), col("constructorId"), col("sum(points)").alias("totalPoints"), denseRankPoints.alias("denseRankPoints"))
dfRankedDrivers.show()

+--------+-------------+-----------+---------------+
|driverId|constructorId|totalPoints|denseRankPoints|
+--------+-------------+-----------+---------------+
|       1|            1|        913|              1|
|      18|            1|        908|              2|
|     117|            1|        458|              3|
|     102|            1|        451|              4|
|      14|            1|        412|              5|
|      57|            1|        407|              6|
|     846|            1|        378|              7|
|       8|            1|        337|              8|
|       4|            1|        241|              9|
|     832|            1|        201|             10|
|      77|            1|        135|             11|
|     817|            1|        130|             12|
|     182|            1|        128|             13|
|     231|            1|        117|             14|
|     187|            1|        109|             15|
|     224|            1|        100|          

In [5]:
# Keep the 3 best drivers of every constructor
dfBestDrivers = dfRankedDrivers.where((col("denseRankPoints")==1) | (col("denseRankPoints")==2) | (col("denseRankPoints")==3))
dfBestDrivers.show()

+--------+-------------+-----------+---------------+
|driverId|constructorId|totalPoints|denseRankPoints|
+--------+-------------+-----------+---------------+
|       1|            1|        913|              1|
|      18|            1|        908|              2|
|     117|            1|        458|              3|
|     815|           10|        400|              1|
|     807|           10|        289|              2|
|     839|           10|        136|              3|
|     376|          100|          0|              1|
|     484|          101|          0|              1|
|     486|          102|          0|              1|
|     475|          102|          0|              1|
|     488|          103|          0|              1|
|     545|          104|          0|              1|
|     418|          104|          0|              1|
|     579|          105|         99|              1|
|     554|          105|         36|              2|
|     475|          105|         33|          

In [6]:
# Join result with drivers and constructors table
joinedDF = dfBestDrivers.join(filteredDriversDF, ['driverId']).join(filteredConstructorsDF, ['constructorId'])

# Delete drivers with 0 points
finalDF = joinedDF.where(col("totalPoints")>0)
finalDF.show()
finalDF.printSchema()

+-------------+--------+-----------+---------------+------------------+------------------+----------------+-----------------------+
|constructorId|driverId|totalPoints|denseRankPoints|       driver_name|driver_nationality|constructor_name|constructor_nationality|
+-------------+--------+-----------+---------------+------------------+------------------+----------------+-----------------------+
|            1|       1|        913|              1|    Lewis Hamilton|           British|         McLaren|                British|
|            1|      18|        908|              2|     Jenson Button|           British|         McLaren|                British|
|            1|     117|        458|              3|       Alain Prost|            French|         McLaren|                British|
|           10|     815|        400|              1|      Sergio Pérez|           Mexican|     Force India|                 Indian|
|           10|     807|        289|              2|   Nico Hülkenberg|     

In [7]:
# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "temp-group1-ass2"  # use your bucket 
spark.conf.set('temporaryGcsBucket', bucket)

# Saving the data to BigQuery
finalDF.write.format('bigquery') \
  .option('table', 'deassignment2.Output_processing_pipeline.bestdrivers') \
  .mode("overwrite") \
  .save()

In [8]:
# Stop the spark context
spark.stop()